In [1]:
import sys
import time
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QWidget, QProgressBar
from PyQt5.QtCore import QThread, pyqtSignal, QObject

class WorkerSignals(QObject):
    finished = pyqtSignal()
    progress = pyqtSignal(int)
    error = pyqtSignal(str)

class Worker(QThread):
    def __init__(self, n):
        super().__init__()
        self.n = n
        self.signals = WorkerSignals()

    def run(self):
        try:
            for i in range(self.n):
                time.sleep(0.1)
                self.signals.progress.emit(int((i + 1) / self.n * 100))
            self.signals.finished.emit()
        except Exception as e:
            self.signals.error.emit(str(e))

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PyQt5 다중 Thread 예제")
        self.setGeometry(100, 100, 400, 300)

        layout = QVBoxLayout()

        self.start_button = QPushButton("작업 시작")
        self.start_button.clicked.connect(self.start_workers)
        layout.addWidget(self.start_button)

        self.progress_bars = []
        for i in range(3):
            progress_bar = QProgressBar()
            layout.addWidget(progress_bar)
            self.progress_bars.append(progress_bar)

        self.status_label = QLabel("대기 중...")
        layout.addWidget(self.status_label)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        self.workers = []

    def start_workers(self):
        self.start_button.setEnabled(False)
        self.status_label.setText("작업 중...")
        
        for i, progress_bar in enumerate(self.progress_bars):
            worker = Worker(100 + i * 50)  # 각 작업의 길이를 다르게 설정
            worker.signals.progress.connect(progress_bar.setValue)
            worker.signals.finished.connect(self.check_workers_finished)
            worker.signals.error.connect(self.handle_error)
            self.workers.append(worker)
            worker.start()

    def check_workers_finished(self):
        if all(not worker.isRunning() for worker in self.workers):
            self.status_label.setText("모든 작업 완료!")
            self.start_button.setEnabled(True)
            self.workers.clear()

    def handle_error(self, error_message):
        self.status_label.setText(f"오류 발생: {error_message}")
        self.start_button.setEnabled(True)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

0 <PyQt5.QtWidgets.QProgressBar object at 0x00000288B22AC5E0>
1 <PyQt5.QtWidgets.QProgressBar object at 0x00000288B22AC670>
2 <PyQt5.QtWidgets.QProgressBar object at 0x00000288B22AC700>


SystemExit: 0

c:\Users\galac\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
